## 线性回归的从零开始实现

In [1]:
%matplotlib inline
import random
import torch
import sys
sys.path.append(r"/Users/kingsleycheng/Documents/github/UCAS/DeepLearning/D2L/")
import mytorch

### 生成数据集

In [2]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = mytorch.synthetic_data(true_w,true_b,1000)


SyntaxError: invalid syntax (410882898.py, line 3)